In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [65]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [67]:
df_add = df_add[:2]


In [72]:
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

doc_list = []
for doc in df_add:

    #구두점 제거
    doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()

    #숫자 제거
    doc2 = "".join([i for i in doc1 if not i.isdigit()])
    
    #월 제거
    month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
            'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
             'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   

    doc3 = " ".join([i for i in doc2.split() if i not in month])
    doc_list.append(doc3)

In [81]:
n_gram_range = (2, 2)
stop_words = "english"

candidates_list = []
for doc in doc_list:
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc3])
    candidates = count.get_feature_names_out()
    candidates_list.append(candidates)

In [102]:
#문서 전체와 문서에서 추출한 키워드 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_bigram_keywords = []
top_n = 20

for i in range(len(doc_list)):
    doc_embedding = model.encode([doc_list[i]])
    candidates_embedding = model.encode(candidates_list[i])
    distances = cosine_similarity(doc_embedding, candidates_embedding)
    doc_bigram_keywords.append([candidates_list[i][index] for index in distances.argsort()[0][-top_n:]])


In [103]:
doc_bigram_keywords[1]

['vehicles australian',
 'duty electric',
 'belarus currently',
 'duty free',
 'certain electric',
 'hydrogen fuelcell',
 'requirement russia',
 'revised tariff',
 'propose tariff',
 'threshold free',
 'electric vehicles',
 'hybrid vehicles',
 'amendments online',
 'vehicles hydrogen',
 'tariff alterations',
 'new tariff',
 'car tax',
 'russia belarus',
 'online tariff',
 'tax threshold']

In [104]:
#문서와 가장 유사한 키워드 벡터화
doc_bigram_embeddings = []
for i in range(len(doc_list)):
    doc_bigram_embedding = []
    for keyword in doc_bigram_keywords[i]:
        doc_bigram_embedding.append(model.encode(keyword))
    doc_bigram_embeddings.append(doc_bigram_embedding)

In [105]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
keyword_embeddings = []

for ele in keyword:
    keyword_embeddings.append(model.encode(ele))

In [108]:
#문서 키워드(doc_bigram_embedding)와 키워드 100(keyword_embedding) 유사도 비교 
bigram_key_result = []
keyword_key_result = []


for index, bigram in enumerate(doc_bigram_embeddings):
    bigram_result = []
    key_result=[]
    
    for i in range(len(bigram)):
        for j in range(len(keyword_embeddings)):
            distances = cosine_similarity([bigram[i]],[keyword_embeddings[j]])
            if distances[0][0] > 0.75:
                bigram_result.append(doc_bigram_keywords[index][i])
                key_result.append(keyword[j])
    bigram_key_result.append(bigram_result)
    keyword_key_result.append(key_result)

In [109]:
print(bigram_key_result)
print(keyword_key_result)

[['car tax'], ['duty electric', 'car tax']]
[['car'], ['propane', 'car']]


In [110]:
df_check = pd.DataFrame()

print(bigram_key_result)
print(keyword_key_result)

# for i in range(len(bigram_key_result)):
for i in range(1):
    bigram = pd.Series(bigram_key_result[i+1])
    keyword = pd.Series(keyword_key_result[i+1])
    
    df_check['bigram'] = bigram
    df_check['keyword'] = keyword

df_check

[['car tax'], ['duty electric', 'car tax']]
[['car'], ['propane', 'car']]


,bigram,keyword
0,duty electric,propane
1,car tax,car
